In [7]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tqdm
import shutil
import warnings
warnings.filterwarnings('ignore')

def show_subplot_img(img_list, size = 10):
    fig = plt.figure(figsize=(size,size))

    for n in range(len(img_list)):
        ax = fig.add_subplot(1,len(img_list),n+1)
        ax.imshow(cv2.cvtColor(img_list[n], cv2.COLOR_BGR2RGB),cmap="gray")
        ax.axis('off')
    plt.show()

def validate_dataset(df, img_dir):
    count = 0
    for rows in df.itertuples():
        if os.path.isfile(img_dir+'/'+ rows.id):
            continue
        else:
            count += 1
            df.drop(df[df['id'] == rows.id].index, inplace=True)
    print("Not founded images (Num) : ",count)
    return df

In [51]:
base_path = r"E:\claassifier/images/"
target_path = r"E:\claassifier/images2/"
os.makedirs(r"E:\claassifier/images2/", exist_ok=True)

df = pd.read_csv(base_path + "result.csv")
df = validate_dataset(df=df, img_dir=base_path)
col_list = df.columns
col_list

Not founded images (Num) :  24


Index(['id', 'good', 'b_edge', 'burr', 'borken', 'b_bubble', 'etc', 'no_lens'], dtype='object')

In [52]:
print(len(df))
df['sum'] = df[['good', 'b_edge', 'burr', 'borken', 'b_bubble', 'etc', 'no_lens']].sum(axis=1)
df.drop(df[df['sum'] == 0].index, inplace=True)
print(len(df))

129851
108823


In [76]:
df

,id,good,b_edge,burr,borken,b_bubble,etc,no_lens,sum
0,01-2.jpg,0,0,0,0,1,0,0,1
2,02-2.jpg,0,0,0,0,1,0,0,1
3,03-2.jpg,0,0,0,0,1,0,0,1
5,04-2.jpg,0,0,0,0,1,0,0,1
6,05-2.jpg,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
129870,None_20230510151857_1_L_1.jpg,0,1,0,0,0,0,0,1
129871,None_20230510151857_2_L_1.jpg,0,1,0,0,0,0,0,1
129872,None_20230510151857_4_L_1.jpg,0,1,0,0,0,0,0,1
129873,None_20230510151910_3_R_1.jpg,0,1,0,0,0,0,0,1


In [53]:
wrong_df = df.loc[(df['good'] == 1) & (df['sum']>1)]
wrong_df


,id,good,b_edge,burr,borken,b_bubble,etc,no_lens,sum


In [54]:
b_edge_df = df[df['b_edge']==1]
burr_df = df[df['burr']==1]
borken_df = df[df['borken']==1]
etc_df = df[df['etc']==1]

print(len(b_edge_df))
print(len(burr_df))
print(len(borken_df))
print(len(etc_df))


786
249
329
2381


In [77]:
new_df = pd.DataFrame(columns=col_list)
save_img = False
be_bar = tqdm.tqdm(df.itertuples(), total=len(df))
for rows in be_bar:
    if rows.b_edge > 0 or rows.burr > 0 or rows.borken > 0 or rows.b_bubble >0:
        new_row = pd.Series([rows.id, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)
        img = cv2.imread(base_path + rows.id)
        if os.path.isfile(target_path + rows.id) is False and save_img:
            shutil.copy(base_path + rows.id, target_path + rows.id)
        
        img_flip_lr = cv2.flip(img, 1)
        lr_filename = 'lr'+rows.id
        if os.path.isfile(target_path + lr_filename) is False and save_img:
            cv2.imwrite(target_path + lr_filename ,img_flip_lr)
        new_row = pd.Series([lr_filename, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)

        img_flip_ud = cv2.flip(img, 0)
        ud_filename = "ud" + rows.id
        if os.path.isfile(target_path + ud_filename) is False and save_img:
            cv2.imwrite(target_path + ud_filename ,img_flip_ud)
        new_row = pd.Series([ud_filename, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)

        img_flip_lrud = cv2.flip(img_flip_lr, 0)
        lrud_filename = 'lrud' + rows.id
        if os.path.isfile(target_path + lrud_filename) is False and save_img:
            cv2.imwrite(target_path + lrud_filename ,img_flip_lrud)
        new_row = pd.Series([lrud_filename, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)
    elif rows.etc > 0:
        new_row = pd.Series([rows.id, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)
        img = cv2.imread(base_path + rows.id)
        if os.path.isfile(target_path + rows.id) is False and save_img:
            shutil.copy(base_path + rows.id, target_path + rows.id)


new_df

  1%|          | 607/108823 [00:16<49:19, 36.56it/s]  


KeyboardInterrupt: 

In [100]:
import random
ok_df = df[df['good']>0]

ok_df = ok_df.reset_index(drop=True)

rand_num_list = random.sample(range(len(ok_df)), 2500)
print(len(rand_num_list))
move_ok_df = ok_df.loc[ok_df.index.isin(rand_num_list)]

m_bar = tqdm.tqdm(move_ok_df.itertuples(), total=len(move_ok_df))
for rows in m_bar:
    new_row = pd.Series([rows.id, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
    new_df = new_df.append(new_row, ignore_index=True)
    if os.path.isfile(target_path + rows.id) is False and save_img:
        shutil.copy(base_path + rows.id, target_path + rows.id)
new_df

2500


100%|██████████| 2500/2500 [00:03<00:00, 832.22it/s]


,id,good,b_edge,burr,borken,b_bubble,etc,no_lens
0,01-2.jpg,0,0,0,0,1,0,0
1,lr01-2.jpg,0,0,0,0,1,0,0
2,ud01-2.jpg,0,0,0,0,1,0,0
3,lrud01-2.jpg,0,0,0,0,1,0,0
4,02-2.jpg,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
4453,None_20230508114726_1_L_1.jpg,1,0,0,0,0,0,0
4454,None_20230508115357_4_L_1.jpg,1,0,0,0,0,0,0
4455,None_20230508115357_4_R_1.jpg,1,0,0,0,0,0,0
4456,None_20230508115448_1_L_1.jpg,1,0,0,0,0,0,0


In [38]:
## 빈이미지 뽑기
import os
import cv2
import numpy as np
import pandas as pd

def cut_roi(img):

    margin = 40 # 여백
    radius = 750
    cy = img.shape[0]//2
    cx = img.shape[1]//2

    x_min = int(cx - radius - margin)
    y_min = int(cy - radius - margin)
    x_max = int(cx + radius + margin)
    y_max = int(cy + radius + margin)
    img_roi = img[y_min:y_max,x_min:x_max]

    return img_roi

def concat_img(white_img, black_img):
    white_img = 255 - white_img

    img_sum = np.zeros((1500,1500,3), dtype=np.uint8)
    img_sum[:,:,0] = cv2.resize(white_img,(1500,1500))
    img_sum[:,:,1] = cv2.resize(black_img,(1500,1500))
    
    return img_sum

def rotation_img(img, div_theta):
    
    div_freq = int(360 / div_theta)

    rimg_list = []

    for i in range(div_freq):
        M = cv2.getRotationMatrix2D(
            (img.shape[0] // 2 ,img.shape[1] // 2),
            div_theta*i,
            1.0
            )
        temp_img = None
        temp_img = cv2.warpAffine(img, M,(img.shape[0],img.shape[1]))
        _, mask = cv2.threshold(temp_img, 0, 1, cv2.THRESH_BINARY_INV)
        temp_img = cv2.copyTo(img, mask, temp_img)

        rimg_list.append(temp_img)
        
    return rimg_list

no_lens_dir =  r"E:\claassifier/no_lens/"
no_lens_target_dir = no_lens_dir + "aug/"
os.makedirs(no_lens_target_dir, exist_ok=True)

nolens_list = [x for x in os.listdir(no_lens_dir) if x.split("_")[-1][0] == "0"] #white이미지만.

print(len(nolens_list))

dddf = pd.read_csv("lens_cls_dataset.csv")
print(dddf.sum())

120
id          01-2.jpglr01-2.jpgud01-2.jpglrud01-2.jpg02-2.j...
good                                                     4947
b_edge                                                   3144
burr                                                      996
borken                                                   1316
b_bubble                                                 5156
etc                                                      2510
no_lens                                                     0
dtype: object


In [40]:
for filename in nolens_list:
    
    w_img = cv2.imread(no_lens_dir + filename,0)
    b_img = cv2.imread(no_lens_dir + "_".join(filename.split("_")[:-1])+ "_1.jpg",0)
    
    w_roi = cut_roi(w_img)
    b_roi = cut_roi(b_img)

    img_sum = concat_img(white_img=w_roi, black_img=b_roi)

    cv2.imwrite(no_lens_target_dir + filename, img_sum)
    new_row = pd.Series([filename,0,0,0,0,0,0,1], index=dddf.columns)
    dddf = dddf.append(new_row, ignore_index=True)

    img_flip_lr = cv2.flip(img_sum, 1)
    lr_filename = 'lr'+ filename
    if os.path.isfile(no_lens_target_dir + lr_filename) is False:
        cv2.imwrite(no_lens_target_dir + lr_filename ,img_flip_lr)
        new_row = pd.Series([lr_filename,0,0,0,0,0,0,1], index=dddf.columns)
        dddf = dddf.append(new_row, ignore_index=True)

    rotated_img_list = rotation_img(img=img_sum, div_theta=20)

    for i , rimg in enumerate(rotated_img_list):
        add_name = "rotate_" + str(i)
        rotate_filename = ".".join(filename.split(".")[:-1]) +add_name+ ".jpg"
        cv2.imwrite(no_lens_target_dir + rotate_filename, rimg)
        new_row = pd.Series([rotate_filename,0,0,0,0,0,0,1], index=dddf.columns)
        dddf = dddf.append(new_row, ignore_index=True)

    frotated_img_list = rotation_img(img=img_flip_lr, div_theta=20)

    for i , rimg in enumerate(frotated_img_list):
        add_name = "frotate_" + str(i)
        frotate_filename = ".".join(lr_filename.split(".")[:-1]) +add_name+ ".jpg"
        cv2.imwrite(no_lens_target_dir + frotate_filename, rimg)
        new_row = pd.Series([frotate_filename,0,0,0,0,0,0,1], index=dddf.columns)
        dddf = dddf.append(new_row, ignore_index=True)
    
print(dddf.sum())

id          01-2.jpglr01-2.jpgud01-2.jpglrud01-2.jpg02-2.j...
good                                                     4947
b_edge                                                   3144
burr                                                      996
borken                                                   1316
b_bubble                                                 5156
etc                                                      2510
no_lens                                                  4598
dtype: object


In [41]:
dddf = dddf.drop_duplicates()
dddf.sum()


id          01-2.jpglr01-2.jpgud01-2.jpglrud01-2.jpg02-2.j...
good                                                     4947
b_edge                                                   3144
burr                                                      996
borken                                                   1316
b_bubble                                                 5156
etc                                                      2510
no_lens                                                  4560
dtype: object

In [42]:
dddf.to_csv("dataset.csv")

In [44]:
g_count = 0
count = 0
for rows in dddf.itertuples():
    if os.path.isfile(no_lens_target_dir+'/'+ rows.id):
        g_count += 1
        continue
    else:
        count += 1
print("Not founded images (Num) : ",count)
print("Not founded images (Num) : ",g_count)


Not founded images (Num) :  17585
Not founded images (Num) :  4560
